In [ ]:
import requests as req
import os, json
from bs4 import BeautifulSoup as bs
import re

# 建立儲存文字檔的資料夾，不存在就新增
folderPath = 'project_gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

url = "https://www.gutenberg.org/browse/languages/zh"
res = req.get(url)
soup = bs(res.text, "html.parser")

list_posts = []
pattern = re.compile(r'/ebooks/(\d+)')
chinese_pattern = re.compile(r'^[\u4e00-\u9fff]+$')  # Ensures the name consists ONLY of Chinese characters

for li in soup.select('div.pgdbbylanguage li.pgdbetext'):
    a = li.select_one('a[href]')
    if a:
        book_id_match = pattern.search(a.get('href'))
        book_name = a.get_text().strip()  # Extract book name
        
        # Apply filtering: Only keep book names that are entirely Chinese characters
        if book_id_match and chinese_pattern.match(book_name):
            book_id = book_id_match.group(1)
            list_posts.append({
                '書籍 ID': book_id,
                '書名': book_name,
                'link': f'https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt'
            })

# Print filtered results
print(json.dumps(list_posts, indent=4, ensure_ascii=False))


for index, obj in enumerate(list_posts):
    res_ = req.get(obj['link'])
    
    chinese_text = re.findall(r'[\u4e00-\u9fff]+', res_.text)  # Extract only Chinese characters
    filtered_text = ' '.join(chinese_text)  # Join extracted characters into a string

    # Corrected filename formatting
    filename = os.path.join(folderPath, f"{obj['書名']}.txt")
    with open(filename, "w", encoding="utf-8") as file:
        file.write(filtered_text)

    print(f"Saved: {filename}")